In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from DCGAN import Discriminator, Generator, initialize_weights

# 训练时的超参数
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LEARING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 3   # RGB
Z_DIM = 100        # 噪声维度
NUM_EPOCH = 5
FEATURES_DISC = 64
FEATURES_GEN = 64


Success!


In [2]:
# 定义图像处理的参数
transforms =torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(IMAGE_SIZE),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)])
    ])

# dataset
dataset = datasets.ImageFolder(root="data/Bangs/", transform=transforms)                 # RGB

loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# 初始化生成器和鉴别器
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)             # （噪声影像的通道数、特征图的通道数）
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)              # （图像的通道数、特征图的通道数）

# 初始化权重参数
initialize_weights(gen)
initialize_weights(disc)

# 定义生成器和鉴别器的优化器
opt_gen = optim.Adam(gen.parameters(), lr=LEARING_RATE, betas=(0.5, 0.999))
opt_disd = optim.Adam(disc.parameters(), lr=LEARING_RATE, betas=(0.5, 0.999))

# 损失函数
criterion = nn.BCELoss()

# 噪声影像
fixed_noise = torch.randn(32, Z_DIM, 1, 1).to(device)

# 生成文件的存储路径
writer_real = SummaryWriter("result/log/CelebFace/Bangs/real")                                       
writer_fake = SummaryWriter("result/log/CelebFace/Bangs/fake")

# 模型改为训练模式（启用BN层）
gen.train()
disc.train()
step = 0

In [3]:
# 开始训练
for epoch in range(NUM_EPOCH):
    for batch_idx, (real, _) in enumerate(loader):                                         # 遍历加载数据
        real = real.to(device)                                                             # 展开到GPU上
        noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)                            # 生成噪声影像
        fake = gen(noise)                                                                  # 利用噪声影像生成的假的影像

        # 训练鉴别器  损失函数：max log(D(x)) + log(1 - D(G(z)))
        disc_real = disc(real).reshape(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))                  # --- 得log(D(real))
        disc_fake = disc(fake.detach()).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))                 # --- 得log(1-D(G(z)))
        loss_disc = (loss_disc_fake + loss_disc_real)/2

        disc.zero_grad()
        loss_disc.backward()
        opt_disd.step()

        # 训练生成器 min log(1-D(G))------ max log(D(G(z)))
        output = disc(fake).reshape(-1)
        loss_gen = criterion(output, torch.ones_like(output))                              # ———— 得log(D(G(z))
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        # 输处损失值等
        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCH}] Batch {batch_idx}/{len(loader)} \
                  Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f} "
            )
            with torch.no_grad():
                fake = gen(fixed_noise)
                # 输出32个示例
                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )
                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)

            step += 1


# 显示结果 tensorboard --logdir="D:\DCGAN_doc\result\log2"

Epoch [0/5] Batch 0/240                   Loss D: 0.6904, loss G: 0.8190 
Epoch [0/5] Batch 100/240                   Loss D: 0.0156, loss G: 4.1481 
Epoch [0/5] Batch 200/240                   Loss D: 0.0082, loss G: 4.7379 
Epoch [1/5] Batch 0/240                   Loss D: 0.0052, loss G: 5.1393 
Epoch [1/5] Batch 100/240                   Loss D: 0.0025, loss G: 5.7877 
Epoch [1/5] Batch 200/240                   Loss D: 0.0015, loss G: 6.2970 
Epoch [2/5] Batch 0/240                   Loss D: 0.0013, loss G: 6.4427 
Epoch [2/5] Batch 100/240                   Loss D: 0.0009, loss G: 6.8716 
Epoch [2/5] Batch 200/240                   Loss D: 0.0006, loss G: 7.2573 
Epoch [3/5] Batch 0/240                   Loss D: 0.0005, loss G: 7.3762 
Epoch [3/5] Batch 100/240                   Loss D: 0.0004, loss G: 7.6401 
Epoch [3/5] Batch 200/240                   Loss D: 0.0003, loss G: 7.9486 
Epoch [4/5] Batch 0/240                   Loss D: 0.0003, loss G: 8.0371 
Epoch [4/5] Batch 100/

In [4]:
# tensorboard --logdir="\result\log\CelebFace\Bangs\"

SyntaxError: cannot assign to operator (3936715073.py, line 1)